# REVIEW > DF로 로딩 함수

In [1]:
import json
import pandas as pd
from dateutil import parser
from datetime import datetime

# file 로딩 > df 만드는 함수
def load_data(data_path):
    # 파일로딩
    with open(f"{data_path}", "r", encoding="utf-8") as file:
        json_data = json.load(file)

    # 로딩 데이터 > df로 만들기
    df = pd.DataFrame(json_data)
    # print(df.head())
    # 데이터 전처리
    df['date'].str.replace(".","-")

    # date 형태(2025.05.06)가 아닌경우, 날짜로 바꾸기
    # 문자열을 날짜로 변환, 오류가 나는 경우 NaT로 처리
    df['date'] = pd.to_datetime(df['date'], errors='coerce')

    # 오늘 날짜
    today = pd.Timestamp(datetime.today().date())

    # NaT인 경우 오늘 날짜로 대체
    df['date'] = df['date'].fillna(today)

    # df 리턴하기 
    return df

In [2]:
datetime.today().date()

datetime.date(2025, 6, 5)

In [3]:
# 함수 호출부
data_path = "res/1.gangneung_mare.json"
df = load_data(data_path)
print(len(df))
df.head()

420


C:\Users\Admin\AppData\Local\Temp\ipykernel_20460\2330629591.py:20: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['date'] = pd.to_datetime(df['date'], errors='coerce')


,review,stars,date
0,"바닷가 바로앞이라 너무좋아요.\n객실도 깨끗하고, 샤워기 수압좋고 따뜻한물 잘 나오...",5,2025-06-05
1,오션뷰라 진짜 좋았어요!! ㅎㅎㅎ\n제 생일이라 남자친구하고 처음 오션뷰 예약해서 ...,5,2025-06-05
2,"황금연휴에 방문했는데 오션뷰가 정말 예쁘더라고요👍\n저는 2층에 머물렀는데, 창밖으...",5,2025-05-06
3,위치좋고 직원들 전부 친절하고 재방문의사100%입니다,5,2025-06-05
4,좋았습니다 수고하세요,5,2025-06-05


# mysql DB 연결하기

In [16]:
from dotenv import load_dotenv
import os
import pandas as pd
from sqlalchemy import create_engine

# .env 파일을 찾아서 환경변수 로딩
load_dotenv('.env', override=True)

# 메모리에 로딩된 환경변수 값을 가져오기
USER = os.environ.get("USER")
PASSWORD = os.environ.get("PASSWORD")
HOST = os.environ.get("HOST")
DB_NAME = os.environ.get("DB_NAME")
PORT = 3306

# DB_RUL = f"mysql+pymsql://{id}:{pw}@{서버ip}:{port}/{db}"
db_url = f"mysql+pymysql://{USER}:{PASSWORD}@{HOST}:{PORT}/{DB_NAME}"

# df > mysql 테이블에 저장

In [17]:
# df -> db save
def save_to_db(df, table_name):
    try:
        # SQLAlchemy 엔진 생성
        engine = create_engine(db_url)

        # DataFrame을 MySQL 테이블에 저장
        df.to_sql(
            name = table_name,
            con=engine,
            if_exists='append',  # 기존 데이터에 추가
            index=False,         # 인덱스는 저장하지 않음
            method='multi'       # 성능 향상을 위한 bulk insert
        )

        print(f"SQLAlchemy를 통해 {len(df)}개의 리뷰가 성공적으로 저장되었습니다.")

    except Exception as e:
        print(f"SQLAlchemy 저장 중 오류 발생: {e}")

## 숙소 정보 DB 저장

In [6]:
import pandas as pd
df = pd.read_csv("res/accommodation_info.csv", encoding='utf-8')
# 숙소정보 DB 저장
table_name = 'accommodation_info'

df.head()
save_to_db(df, table_name)
# print(f"{table_name}에 데이터 저장 완료")

SQLAlchemy를 통해 4개의 리뷰가 성공적으로 저장되었습니다.


## 리뷰데이터 저장하기

In [7]:
# 전처리
data_path = "res/1.gangneung_mare.json"
df = load_data(data_path)

# df에 accommodation_id 추가하기
# 강릉 마레는 accommodation_id = 1
df['accommodation_id'] = 1
# df.head()

# DB 테이블에 저장하기
table_name = "accommodation_review"
save_to_db(df, table_name)

# save_to_db(df, "accommodation_review")
# save_to_db(df, table_name)

SQLAlchemy를 통해 420개의 리뷰가 성공적으로 저장되었습니다.


C:\Users\Admin\AppData\Local\Temp\ipykernel_20460\2330629591.py:20: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['date'] = pd.to_datetime(df['date'], errors='coerce')


## 여러개의 파일을 일괄로 전처리하기

- 함수정의

In [8]:
import json
import pandas as pd
from dateutil import parser
from datetime import datetime

# file 로딩 > df 만드는 함수
def load_data_multi(acco_id, data_path):
    # 파일로딩
    with open(f"{data_path}", "r", encoding="utf-8") as file:
        json_data = json.load(file)

    # 로딩 데이터 > df로 만들기
    df = pd.DataFrame(json_data)
    # print(df.head())
    # 데이터 전처리
    df['date'].str.replace(".","-")

    # date 형태(2025.05.06)가 아닌경우, 날짜로 바꾸기
    # 문자열을 날짜로 변환, 오류가 나는 경우 NaT로 처리
    df['date'] = pd.to_datetime(df['date'], errors='coerce')

    # 오늘 날짜
    today = pd.Timestamp(datetime.today().date())

    # NaT인 경우 오늘 날짜로 대체
    df['date'] = df['date'].fillna(today)
    
    # df 에 accommodation_id 추가하기
    # 강릉 마레는 accommodation_id =1
    df['accommodation_id'] = 1

    # df 리턴하기 
    return df

- 일괄실행

In [9]:
# 전처리하고 저장해야하는 데이터 파일이 10
MAPPING = {
    # 1: './res/1.gangneung_mare.json',
    2: './res/2.insadong_ninetree.json',
    3: './res/3.kyongpo_the_hongc.json',
    # 4: './res/4.hongcheon_vivaldi.json'
}
for acco_id, data_path in MAPPING.items():
# 전처리
    # data_path = "res/1.gangneung_mare.json"
    df = load_data_multi(acco_id, data_path)

    # DB 테이블에 저장하기
    table_name = "accommodation_review"
    save_to_db(df, table_name)


SQLAlchemy를 통해 400개의 리뷰가 성공적으로 저장되었습니다.
SQLAlchemy를 통해 420개의 리뷰가 성공적으로 저장되었습니다.


C:\Users\Admin\AppData\Local\Temp\ipykernel_20460\2825726136.py:20: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['date'] = pd.to_datetime(df['date'], errors='coerce')


In [10]:
# models.py 모듈을 인식하기 위해 경로 추가,
import sys
sys.path.append('C:/Users/Admin/kpmg_future_labyanolja_summary/data_prepare')  # 실제 경로로 변경

In [11]:
from summary_modules import load_prompt, preprocess_reviews, summarize

In [12]:

prompt_file = "res/prompt_1shotv1.pickle"
prompt = load_prompt(prompt_file)
prompt

'당신은 요약 전문가입니다. 사용자 숙소 리뷰들이 주어졌을 때 요약하는 것이 당신의 목표입니다. 다음은 리뷰들과 요약 예시입니다.\n예시 리뷰들:\n[REVIEW_START]강릉 여행길에  들럿는데  아주 예약을 잘했어요\n호텔이  주인의 품격이 느껴지더군요  \n조용하고 좋습니다[REVIEW_END]\n[REVIEW_START]로비도 고급지고 객실도 깔끔하니 너무나 만족스런1박 했습니다.\n침구가 너무 좋아서 잘 잘수있었어요.\n다음에 여행가면 여기에 다시 묵을겁니다[REVIEW_END]\n[REVIEW_START]친절한 안내와 정리정돈도 잘 되어 있어 편안하게 잘수있었어요[REVIEW_END]\n[REVIEW_START]부모님 여행 숙소 예약해드렸는데 너무 만족하셨어요! 굿![REVIEW_END]\n[REVIEW_START]자주 갔던곳인데 너무 깨끗하고 아늑하고 좋아요\n침구류 냄새 하도 안나서 좋았습니다[REVIEW_END]\n[REVIEW_START]밖에 나와서 주치 안내해 준 직원분에게 감사인사를 전합니다.  직원분들 모두 친절하시네요[REVIEW_END]\n[REVIEW_START]객실 깨끗하고 편하고 다 좋은데\n패스트푸드제외\n배달음식 못시켜먹는것이 아쉽습니다\n직원분들 너무친절하시고 좋아요!![REVIEW_END]\n[REVIEW_START]시설 깔끔하고 택지지구에 있어 주변 인프라가 좋습니다 다만 외부 음식 관련 안내는 사전 안내가 필요할 것 같습니다[REVIEW_END]\n[REVIEW_START]프론트분들도 친절하고 방도 깨꿋하고 업무차 강룽가서 처음이용했는데 너무 편하고 좋았어요[REVIEW_END]\n[REVIEW_START]부모님 예약해드렸는데 편안하게 잘 쉬었다고 하시네요! 다만 윗층에 아이들 소리 들린다는 거 보면 방음은 조금 아쉬운 편인거 같아요ㅠㅠ[REVIEW_END]\n[REVIEW_START]편하게 잘 보냈습니다. 조식 샌드위치 진짜 맛있어요!![REVIEW_END]\n[REVIEW_START]직원분들 너무 친절하시고 조식

In [13]:
data_path = "res/1.gangneung_mare.json"
review_good, review_bad = preprocess_reviews(data_path)
summary_good = summarize(review_good, prompt)
summary_bad = summarize(review_bad, prompt)

# 모든 리뷰 요약 데이터 일괄 처리
- 전처리, df 만들기, mysql에 저장

In [14]:
# 전처리하고 저장해야하는 데이터 파일이 10
MAPPING = {
    1: './res/1.gangneung_mare.json',
    2: './res/2.insadong_ninetree.json',
    3: './res/3.kyongpo_the_hongc.json',
    # 4: './res/4.hongcheon_vivaldi.json'
    }

In [18]:

table_name='review_summary'

# 리뷰 전처리
for id, path in MAPPING.items():
    review_good, review_bad = preprocess_reviews(path)

    if review_good:
        summary_good = summarize(review_good, prompt)
        # print(summary_good)
    else:
        summary_good = "분석할 높은 평점 리뷰가 없습니다."
    # 나쁜 리뷰 요약
    if review_bad:
        summary_bad = summarize(review_bad, prompt)
        # print(summary_bad)
    else:
        summary_bad = "분석할 낮은 평점 리뷰가 없습니다."

    # 처리결과 데이터 프레임으로 만들어서 저장하기
    df = pd.DataFrame({
        'accommodation_id': [id],
        'summary_good': [summary_good],
        'summary_bad': [summary_bad],
        'date': [pd.Timestamp.now().strftime('%Y-%m-%d')]
    })
    print(df)

    # # MySQL에 저장    
    save_to_db(df, table_name)
    print(f"{path} 숙소의 리뷰 요약이 완료되었습니다.")

   accommodation_id                                       summary_good  \
0                 1  바닷가 바로 앞에 위치한 숙소로, 객실이 깨끗하고 샤워기 수압이 좋아서 친구들과 함...   

                                         summary_bad        date  
0  바람이 불어 예정보다 30분 늦게 숙소 입실을 허락해줬지만 오션뷰가 기대에 못 미치...  2025-06-05  
SQLAlchemy를 통해 1개의 리뷰가 성공적으로 저장되었습니다.
./res/1.gangneung_mare.json 숙소의 리뷰 요약이 완료되었습니다.
   accommodation_id                                       summary_good  \
0                 2  도심지에 위치한 숙소로 인사동이나 근처를 구경하기에 좋았습니다. 객실이 깨끗하고 안...   

                                         summary_bad        date  
0  청소상태가 불결하고 벽과 세면대에 머리카락이 투성이인 호텔이었습니다. 방음이 전혀 ...  2025-06-05  
SQLAlchemy를 통해 1개의 리뷰가 성공적으로 저장되었습니다.
./res/2.insadong_ninetree.json 숙소의 리뷰 요약이 완료되었습니다.
   accommodation_id                                       summary_good  \
0                 3  강릉 여행길에 들렀을 때 아주 예약을 잘한 호텔이었습니다. 주인의 품격이 느껴지며 ...   

                                         summary_bad        date  
0  예약할 때부터 공사 안내가 필요하다고 생각합니다. 체크인 시에도 공사로 